# Pydantic-based Mesh Example

This notebook demonstrates how to use the new Pydantic-based Mesh class in meshly. It covers:

1. Creating custom Mesh subclasses with additional attributes
2. Working with numpy arrays in Pydantic models
3. Encoding and decoding meshes to/from zip files
4. Optimizing meshes with the built-in optimization methods

In [27]:
from pathlib import Path

import numpy as np

# Import the Mesh class
from meshly import Mesh
from pydantic import Field

## 1. Creating a Custom Mesh Subclass

One of the key benefits of the new Pydantic-based Mesh class is the ability to create custom subclasses with additional attributes. Let's create a `TexturedMesh` class that adds texture coordinates and normals.

In [28]:
from meshly import Packable
from pydantic import BaseModel, ConfigDict


class MaterialProperties(BaseModel):
    """Material properties with numpy arrays - demonstrates BaseModel in dict edge case."""
    model_config = ConfigDict(arbitrary_types_allowed=True)

    name: str = Field(..., description="Material name")
    diffuse: np.ndarray = Field(..., description="Diffuse color array")
    specular: np.ndarray = Field(..., description="Specular color array")
    shininess: float = Field(32.0, description="Shininess value")


class PhysicsProperties(Packable):
    """Physics properties as a nested Packable - demonstrates Packable field support."""
    mass: float = Field(1.0, description="Object mass")
    friction: float = Field(0.5, description="Friction coefficient")
    # Arrays in nested Packable are encoded/decoded using the Packable's own encode/decode
    inertia_tensor: np.ndarray = Field(..., description="3x3 inertia tensor")
    collision_points: np.ndarray = Field(..., description="Collision sample points")


class TexturedMesh(Mesh):
    """
    A mesh with texture coordinates and normals.
    
    This demonstrates how to create a custom Mesh subclass with additional
    numpy array attributes that will be automatically encoded/decoded.
    """
    # Add texture coordinates and normals as additional numpy arrays
    texture_coords: np.ndarray = Field(..., description="Texture coordinates")
    normals: np.ndarray | None = Field(None, description="Vertex normals")

    # Add non-array attributes
    material_name: str = Field("default", description="Material name")
    tags: list[str] = Field(default_factory=list, description="Tags for the mesh")

    # Dictionary containing nested dictionaries with arrays
    material_data: dict[str, dict[str, np.ndarray]] = Field(
        default_factory=dict,
        description="Nested dictionary structure with arrays"
    )

    material_colors: dict[str, str] = Field(
        default_factory=dict,
        description="Dictionary with non-array values"
    )

    # Dictionary containing BaseModel instances with numpy arrays
    # This demonstrates the edge case of dict[str, BaseModel] where BaseModel has arrays
    materials: dict[str, MaterialProperties] = Field(
        default_factory=dict,
        description="Dictionary of material name to MaterialProperties (BaseModel with arrays)"
    )

## 2. Creating a Mesh Instance

Now let's create a simple cube mesh with texture coordinates and normals.

In [29]:
# Create vertices for a cube
vertices = np.array([
    [-0.5, -0.5, -0.5],  # 0: bottom-left-back
    [0.5, -0.5, -0.5],   # 1: bottom-right-back
    [0.5, 0.5, -0.5],    # 2: top-right-back
    [-0.5, 0.5, -0.5],   # 3: top-left-back
    [-0.5, -0.5, 0.5],   # 4: bottom-left-front
    [0.5, -0.5, 0.5],    # 5: bottom-right-front
    [0.5, 0.5, 0.5],     # 6: top-right-front
    [-0.5, 0.5, 0.5]     # 7: top-left-front
], dtype=np.float32)

# Create indices for the cube
indices = np.array([
    [0, 1, 2, 2, 3, 0],  # back face
    [1, 5, 6, 6, 2, 1],  # right face
    [5, 4, 7, 7, 6, 5],  # front face
    [4, 0, 3, 3, 7, 4],  # left face
    [3, 2, 6, 6, 7, 3],  # top face
    [4, 5, 1, 1, 0, 4]   # bottom face
], dtype=np.uint32)

# Create texture coordinates (one for each vertex)
texture_coords = np.array([
    [0.0, 0.0],  # 0
    [1.0, 0.0],  # 1
    [1.0, 1.0],  # 2
    [0.0, 1.0],  # 3
    [0.0, 0.0],  # 4
    [1.0, 0.0],  # 5
    [1.0, 1.0],  # 6
    [0.0, 1.0]   # 7
], dtype=np.float32)

# Create normals (one for each vertex)
normals = np.array([
    [0.0, 0.0, -1.0],  # 0: back
    [0.0, 0.0, -1.0],  # 1: back
    [0.0, 0.0, -1.0],  # 2: back
    [0.0, 0.0, -1.0],  # 3: back
    [0.0, 0.0, 1.0],   # 4: front
    [0.0, 0.0, 1.0],   # 5: front
    [0.0, 0.0, 1.0],   # 6: front
    [0.0, 0.0, 1.0]    # 7: front
], dtype=np.float32)

# Create MaterialProperties instances (BaseModel with numpy arrays)
cube_material = MaterialProperties(
    name="cube_material",
    diffuse=np.array([1.0, 0.5, 0.31], dtype=np.float32),
    specular=np.array([0.5, 0.5, 0.5], dtype=np.float32),
    shininess=32.0
)

secondary_material = MaterialProperties(
    name="secondary_material",
    diffuse=np.array([0.2, 0.8, 0.2], dtype=np.float32),
    specular=np.array([0.3, 0.3, 0.3], dtype=np.float32),
    shininess=16.0
)

# Create PhysicsProperties instance (nested Packable)
physics = PhysicsProperties(
    mass=2.5,
    friction=0.7,
    inertia_tensor=np.eye(3, dtype=np.float32) * 0.1,  # 3x3 identity scaled
    collision_points=np.array([
        [-0.5, -0.5, -0.5],
        [0.5, 0.5, 0.5],
        [0.0, 0.0, 0.0]
    ], dtype=np.float32)
)

# Create the textured mesh
mesh = TexturedMesh(
    vertices=vertices,
    indices=indices,
    texture_coords=texture_coords,
    normals=normals,
    material_name="cube_material",
    tags=["cube", "example"],
    material_data={
        "cube_material": {
            "diffuse": np.array([1.0, 0.5, 0.31], dtype=np.float32),
            "specular": np.array([0.5, 0.5, 0.5], dtype=np.float32),
            "shininess": np.array([32.0], dtype=np.float32)
        }
    },
    material_colors={
        "cube_material": "#FF7F50"
    },
    # dict[str, BaseModel] with numpy arrays inside
    materials={
        "cube_material": cube_material,
        "secondary_material": secondary_material
    },
    # Nested Packable field
    physics=physics
)

print(f"Mesh created with {mesh.vertex_count} vertices and {mesh.index_count} indices")
print(f"Material name: {mesh.material_name}")
print(f"Tags: {mesh.tags}")
print(f"Materials (BaseModel dict): {list(mesh.materials.keys())}")

Mesh created with 8 vertices and 36 indices
Material name: cube_material
Tags: ['cube', 'example']
Materials (BaseModel dict): ['cube_material', 'secondary_material']


## 3. Optimizing the Mesh

The Mesh class provides several optimization methods that can be used to improve rendering performance.

In [30]:
# Optimize the mesh for vertex cache
vertex_cache_optimized_mesh = mesh.optimize_vertex_cache()
print("Optimized for vertex cache")

# Optimize the mesh for overdraw
overdraw_optimized_mesh = mesh.optimize_overdraw()
print("Optimized for overdraw")

# Optimize the mesh for vertex fetch
vertex_fetch_optimized = mesh.optimize_vertex_fetch()
print("Optimized for vertex fetch")

Optimized for vertex cache
Optimized for overdraw
Optimized for vertex fetch


## 4. Encoding and Saving the Mesh

The Mesh class provides methods for encoding the mesh and saving it to a zip file.

In [31]:
# Save the mesh to a zip file
zip_path = Path("textured_cube.zip")
mesh.save_to_zip(zip_path)
assert zip_path.exists()
print(f"Saved mesh to {zip_path} has {mesh.vertex_count} vertices and {mesh.index_count} indices")


decoded_mesh = Mesh.decode(mesh.encode())
print(f"Decoded mesh has {decoded_mesh.vertex_count} vertices and {decoded_mesh.index_count} indices")

Saved mesh to textured_cube.zip has 8 vertices and 36 indices
Decoded mesh has 8 vertices and 36 indices


## 5. Loading the Mesh from a Zip File

The Mesh class provides a class method for loading a mesh from a zip file.

In [32]:
# Load the mesh from the zip file
loaded_mesh = TexturedMesh.load_from_zip(zip_path)
print(f"Loaded mesh: {loaded_mesh.vertex_count} vertices, {loaded_mesh.index_count} indices")
print(f"Material name: {loaded_mesh.material_name}")
print(f"Tags: {loaded_mesh.tags}")

# Verify that the texture coordinates and normals were loaded correctly
print(f"\nTexture coordinates shape: {loaded_mesh.texture_coords.shape}")
print(f"Normals shape: {loaded_mesh.normals.shape}")
print(f"Material data: {loaded_mesh.material_data}")
print(f"Material colors: {loaded_mesh.material_colors}")

# Verify the dict[str, BaseModel] edge case was loaded correctly
print("\n--- BaseModel dict edge case ---")
print(f"Materials keys: {list(loaded_mesh.materials.keys())}")
for mat_name, mat in loaded_mesh.materials.items():
    print(f"  {mat_name}:")
    print(f"    type: {type(mat).__name__}")
    print(f"    diffuse: {mat.diffuse}")
    print(f"    specular: {mat.specular}")
    print(f"    shininess: {mat.shininess}")


Loaded mesh: 8 vertices, 36 indices
Material name: cube_material
Tags: ['cube', 'example']

Texture coordinates shape: (8, 2)
Normals shape: (8, 3)
Material data: {'cube_material': {'diffuse': array([1.  , 0.5 , 0.31], dtype=float32), 'shininess': array([32.], dtype=float32), 'specular': array([0.5, 0.5, 0.5], dtype=float32)}}
Material colors: {'cube_material': '#FF7F50'}

--- BaseModel dict edge case ---
Materials keys: ['cube_material', 'secondary_material']
  cube_material:
    type: MaterialProperties
    diffuse: [1.   0.5  0.31]
    specular: [0.5 0.5 0.5]
    shininess: 32.0
  secondary_material:
    type: MaterialProperties
    diffuse: [0.2 0.8 0.2]
    specular: [0.3 0.3 0.3]
    shininess: 16.0


## 6. Creating a Different Mesh Subclass

Let's create another mesh subclass with different attributes to demonstrate the flexibility of the Pydantic-based Mesh class.

In [33]:
class SkinnedMesh(Mesh):
    """
    A mesh with skinning information for animation.
    """
    # Add bone weights and indices as additional numpy arrays
    bone_weights: np.ndarray = Field(..., description="Bone weights for each vertex")
    bone_indices: np.ndarray = Field(..., description="Bone indices for each vertex")

    # Add non-array attributes
    skeleton_name: str = Field("default", description="Skeleton name")
    animation_names: list[str] = Field(default_factory=list, description="Animation names")

# Create a simple skinned mesh
skinned_mesh = SkinnedMesh(
    vertices=vertices,
    indices=indices,
    bone_weights=np.random.random((len(vertices), 4)).astype(np.float32),  # 4 weights per vertex
    bone_indices=np.random.randint(0, 4, (len(vertices), 4)).astype(np.uint8),  # 4 bone indices per vertex
    skeleton_name="human_skeleton",
    animation_names=["walk", "run", "jump"]
)

print(f"Skinned mesh created with {skinned_mesh.vertex_count} vertices and {skinned_mesh.index_count} indices")
print(f"Skeleton name: {skinned_mesh.skeleton_name}")
print(f"Animation names: {skinned_mesh.animation_names}")
print(f"Bone weights shape: {skinned_mesh.bone_weights.shape}")
print(f"Bone indices shape: {skinned_mesh.bone_indices.shape}")

Skinned mesh created with 8 vertices and 36 indices
Skeleton name: human_skeleton
Animation names: ['walk', 'run', 'jump']
Bone weights shape: (8, 4)
Bone indices shape: (8, 4)


## 7. Saving and Loading the Skinned Mesh

Let's save and load the skinned mesh to demonstrate that all attributes are preserved.

In [34]:
# Save the skinned mesh to a zip file
skinned_zip_path = Path("skinned_cube.zip")
skinned_mesh.save_to_zip(skinned_zip_path)
print(f"Saved skinned mesh to {skinned_zip_path}, file size: {skinned_zip_path.stat().st_size} bytes")

# Load the skinned mesh from the zip file
loaded_skinned_mesh = SkinnedMesh.load_from_zip(skinned_zip_path)
print(f"\nLoaded skinned mesh: {loaded_skinned_mesh.vertex_count} vertices, {loaded_skinned_mesh.index_count} indices")
print(f"Skeleton name: {loaded_skinned_mesh.skeleton_name}")
print(f"Animation names: {loaded_skinned_mesh.animation_names}")
print(f"Bone weights shape: {loaded_skinned_mesh.bone_weights.shape}")
print(f"Bone indices shape: {loaded_skinned_mesh.bone_indices.shape}")

Saved skinned mesh to skinned_cube.zip, file size: 2498 bytes

Loaded skinned mesh: 8 vertices, 36 indices
Skeleton name: human_skeleton
Animation names: ['walk', 'run', 'jump']
Bone weights shape: (8, 4)
Bone indices shape: (8, 4)


## 8. Cleaning Up (Part 1)

Let's clean up the files we created so far before the cache examples.

In [35]:
# Clean up files from sections 1-7
for path in [zip_path, skinned_zip_path]:
    if Path(path).exists():
        Path(path).unlink()
        print(f"Removed {path}")

print("\nSections 1-8 completed!")

Removed textured_cube.zip
Removed skinned_cube.zip

Sections 1-8 completed!


## 9. Nested Packables with Direct Fields

Now that Packables support nested Packable fields directly, let's demonstrate saving and loading a mesh with a nested `PhysicsProperties` Packable.

In [36]:
# Create a mesh subclass with a nested Packable field
class MeshWithPhysics(Mesh):
    """A mesh with embedded physics properties as a nested Packable."""
    physics: PhysicsProperties | None = Field(None, description="Physics properties")
    label: str = Field("default", description="Mesh label")

# Create physics properties
physics_props = PhysicsProperties(
    mass=5.0,
    friction=0.3,
    inertia_tensor=np.diag([1.0, 2.0, 3.0]).astype(np.float32),
    collision_points=vertices.copy()  # Use cube vertices as collision points
)

# Create mesh with nested Packable
mesh_with_physics = MeshWithPhysics(
    vertices=vertices,
    indices=indices,
    physics=physics_props,
    label="physics_cube"
)

print(f"Created mesh with nested physics Packable")
print(f"  Mass: {mesh_with_physics.physics.mass}")
print(f"  Friction: {mesh_with_physics.physics.friction}")
print(f"  Inertia tensor shape: {mesh_with_physics.physics.inertia_tensor.shape}")

Created mesh with nested physics Packable
  Mass: 5.0
  Friction: 0.3
  Inertia tensor shape: (3, 3)


In [37]:
# Save and load the mesh with nested Packable
physics_zip_path = Path("./mesh_with_physics.zip")
mesh_with_physics.save_to_zip(physics_zip_path)

loaded_physics_mesh = MeshWithPhysics.load_from_zip(physics_zip_path)

print("Loaded mesh with nested Packable:")
print(f"  Label: {loaded_physics_mesh.label}")
print(f"  Vertices: {loaded_physics_mesh.vertex_count}")
print(f"  Physics mass: {loaded_physics_mesh.physics.mass}")
print(f"  Physics friction: {loaded_physics_mesh.physics.friction}")
print(f"  Inertia tensor:\n{loaded_physics_mesh.physics.inertia_tensor}")
np.testing.assert_array_almost_equal(
    loaded_physics_mesh.physics.collision_points, 
    physics_props.collision_points
)
print("✓ Collision points match!")

Loaded mesh with nested Packable:
  Label: physics_cube
  Vertices: 8
  Physics mass: 5.0
  Physics friction: 0.3
  Inertia tensor:
[[1. 0. 0.]
 [0. 2. 0.]
 [0. 0. 3.]]
✓ Collision points match!


In [47]:
# Using assets parameter to externalize nested Packables
# This is useful for external storage, caching, or deduplication
import zipfile

# Save with assets dict - nested Packables are stored externally (not in zip)
external_assets: dict[str, bytes] = {}
physics_zip_external = Path("./mesh_physics_external.zip")
mesh_with_physics.save_to_zip(physics_zip_external, assets=external_assets)

print(f"Saved mesh to {physics_zip_external.absolute()}")
print(f"  Zip file size: {physics_zip_external.stat().st_size} bytes")

# Show what's in the zip - note: NO packables/ folder!
with zipfile.ZipFile(physics_zip_external) as zf:
    print(f"  Files in zip: {zf.namelist()}")
    has_packables = any('packables/' in n for n in zf.namelist())
    print(f"  Has embedded packables/: {has_packables}")

print(f"\nExternal assets collected: {len(external_assets)}")
for checksum, data in external_assets.items():
    print(f"  {checksum[:16]}...: {len(data)} bytes (physics Packable)")

Saved mesh to /workspaces/studio/framework/meshlyP/python/mesh_physics_external.zip
  Zip file size: 1483 bytes
  Files in zip: ['arrays/cell_types/array.bin', 'arrays/cell_types/metadata.json', 'arrays/index_sizes/array.bin', 'arrays/index_sizes/metadata.json', 'vertices.bin', 'indices.bin', 'metadata.json']
  Has embedded packables/: False

External assets collected: 1
  4b9ba04bbd1b4482...: 1086 bytes (physics Packable)


In [48]:
# Load using the assets dict as provider
loaded_external = MeshWithPhysics.load_from_zip(physics_zip_external, assets=external_assets)
print(f"Loaded with dict provider:")
print(f"  Physics mass: {loaded_external.physics.mass}")
print(f"  Physics friction: {loaded_external.physics.friction}")

# Load using a callable provider (simulates remote fetch)
fetch_count = [0]
def fetch_asset(checksum: str) -> bytes:
    fetch_count[0] += 1
    print(f"  Fetching {checksum[:16]}...")
    return external_assets[checksum]

loaded_lazy = MeshWithPhysics.load_from_zip(physics_zip_external, assets=fetch_asset)
print(f"\nLoaded with callable provider:")
print(f"  Fetches triggered: {fetch_count[0]}")
print(f"  Physics mass: {loaded_lazy.physics.mass}")

# Clean up
physics_zip_external.unlink()
print(f"\n✓ External assets demo complete!")

Loaded with dict provider:
  Physics mass: 5.0
  Physics friction: 0.3
  Fetching 4b9ba04bbd1b4482...

Loaded with callable provider:
  Fetches triggered: 1
  Physics mass: 5.0

✓ External assets demo complete!


## 10. Extract/Reconstruct with Nested Packables

The `extract()` and `reconstruct()` methods handle nested Packables by encoding them as assets with checksum references. This is useful for:
- Serializing to JSON (data) + binary blobs (assets)
- Remote execution where assets are transferred separately
- Caching and deduplication of shared Packables

In [40]:
# Create a Pydantic model that contains a Mesh (Packable)
class SceneObject(BaseModel):
    """A scene object containing a mesh and transform data."""
    model_config = ConfigDict(arbitrary_types_allowed=True)
    
    name: str = Field(..., description="Object name")
    mesh: Mesh = Field(..., description="The mesh geometry")
    position: np.ndarray = Field(..., description="Position in 3D space")
    rotation: np.ndarray = Field(..., description="Rotation (quaternion)")
    scale: float = Field(1.0, description="Uniform scale")

# Create a scene object with a mesh
scene_obj = SceneObject(
    name="my_cube",
    mesh=Mesh(vertices=vertices, indices=indices),
    position=np.array([1.0, 2.0, 3.0], dtype=np.float32),
    rotation=np.array([0.0, 0.0, 0.0, 1.0], dtype=np.float32),  # identity quaternion
    scale=2.0
)

print(f"Created SceneObject '{scene_obj.name}' with mesh ({scene_obj.mesh.vertex_count} verts)")

Created SceneObject 'my_cube' with mesh (8 verts)


In [41]:
# Extract the scene object - this separates JSON-serializable data from binary assets
extracted = Packable.extract(scene_obj)

print("Extracted data (JSON-serializable):")
print(f"  Keys: {list(extracted.data.keys())}")
print(f"  name: {extracted.data['name']}")
print(f"  scale: {extracted.data['scale']}")
print(f"  mesh: {extracted.data['mesh']}")  # This is a $ref to the encoded Packable
print(f"  position: {extracted.data['position']}")  # This is a $ref to the encoded array

print(f"\nAssets (binary blobs):")
print(f"  Number of assets: {len(extracted.assets)}")
for checksum, data in extracted.assets.items():
    print(f"  {checksum[:16]}...: {len(data)} bytes")

Extracted data (JSON-serializable):
  Keys: ['name', 'mesh', 'position', 'rotation', 'scale']
  name: my_cube
  scale: 2.0
  mesh: {'$ref': '1699137d50666dbe'}
  position: {'$ref': '21849fc3b8888dd5'}

Assets (binary blobs):
  Number of assets: 3
  1699137d50666dbe...: 1416 bytes
  21849fc3b8888dd5...: 123 bytes
  1f9da9f6ca9a12cb...: 123 bytes


In [42]:
import json

# The data dict is JSON-serializable (can be sent over network, stored in DB, etc.)
json_str = json.dumps(extracted.data, indent=2)
print("JSON representation of extracted data:")
print(json_str)

JSON representation of extracted data:
{
  "name": "my_cube",
  "mesh": {
    "$ref": "1699137d50666dbe"
  },
  "position": {
    "$ref": "21849fc3b8888dd5"
  },
  "rotation": {
    "$ref": "1f9da9f6ca9a12cb"
  },
  "scale": 2.0
}


In [43]:
# Reconstruct the scene object from data + assets
# The Pydantic schema tells reconstruct() that 'mesh' is a Mesh type,
# so it knows to decode the Packable bytes back into a Mesh instance
reconstructed = Packable.reconstruct(SceneObject, extracted.data, extracted.assets)

print("Reconstructed SceneObject:")
print(f"  name: {reconstructed.name}")
print(f"  scale: {reconstructed.scale}")
print(f"  position: {reconstructed.position}")
print(f"  mesh type: {type(reconstructed.mesh).__name__}")
print(f"  mesh vertices: {reconstructed.mesh.vertex_count}")

# Verify the data matches
np.testing.assert_array_almost_equal(reconstructed.position, scene_obj.position)
np.testing.assert_array_almost_equal(reconstructed.mesh.vertices, scene_obj.mesh.vertices)
print("\n✓ Reconstructed data matches original!")

Reconstructed SceneObject:
  name: my_cube
  scale: 2.0
  position: [1. 2. 3.]
  mesh type: Mesh
  mesh vertices: 8

✓ Reconstructed data matches original!


### Lazy Loading with Callbacks

When using a callable instead of a dict for assets, `reconstruct()` returns a `LazyModel` that only fetches assets when fields are accessed. This is useful for large meshes or remote storage.

In [44]:
# Simulate a remote asset fetcher with logging
fetch_log = []

def asset_fetcher(checksum: str) -> bytes:
    """Simulates fetching from remote storage."""
    fetch_log.append(checksum)
    print(f"  Fetching asset {checksum[:16]}...")
    return extracted.assets[checksum]

# Reconstruct with callable - returns LazyModel
print("Creating lazy model (no assets fetched yet)...")
lazy_scene = Packable.reconstruct(SceneObject, extracted.data, asset_fetcher)
print(f"  Type: {type(lazy_scene).__name__}")
print(f"  Fetches so far: {len(fetch_log)}")

# Access scalar field - no fetch needed
print(f"\nAccessing 'name': {lazy_scene.name}")
print(f"  Fetches so far: {len(fetch_log)}")

# Access array field - triggers fetch
print(f"\nAccessing 'position':")
pos = lazy_scene.position
print(f"  Value: {pos}")
print(f"  Fetches so far: {len(fetch_log)}")

# Access mesh field - triggers fetch of the Packable
print(f"\nAccessing 'mesh':")
m = lazy_scene.mesh
print(f"  Type: {type(m).__name__}, vertices: {m.vertex_count}")
print(f"  Total fetches: {len(fetch_log)}")

Creating lazy model (no assets fetched yet)...
  Type: LazyModel
  Fetches so far: 0

Accessing 'name': my_cube
  Fetches so far: 0

Accessing 'position':
  Fetching asset 21849fc3b8888dd5...
  Value: [1. 2. 3.]
  Fetches so far: 1

Accessing 'mesh':
  Fetching asset 1699137d50666dbe...
  Type: Mesh, vertices: 8
  Total fetches: 2


In [45]:
# Clean up the physics mesh file
if physics_zip_path.exists():
    physics_zip_path.unlink()
    print(f"Removed {physics_zip_path}")

Removed mesh_with_physics.zip


## 11. Using Callbacks for Nested Packables

When working with meshes that contain nested Packables (like our `TexturedMesh` with `PhysicsProperties`), you can use callbacks to implement custom caching, storage, or deduplication strategies.

The callback types are:
- **`on_packable` (save)**: `Callable[[Packable, str], None]` - called with `(packable, checksum)` when saving
- **`on_packable` (load)**: `Callable[[Type[Packable], str], Optional[Packable]]` - called with `(packable_type, checksum)` when loading; return `None` to fall back to embedded data

### Cache Deduplication Example

When multiple meshes share the same nested Packable data, the cache automatically deduplicates them using SHA256 hashes.

## 12. Jax Conversion Example

In [46]:
try:
    import jax

    jax_skinned_mesh = skinned_mesh.convert_to("jax")
    print(f"Converted skinned mesh to JAX arrays, vertex dtype: {jax_skinned_mesh.vertices.dtype}")
except ImportError:
    print("JAX not available - skipping conversion example")

Converted skinned mesh to JAX arrays, vertex dtype: float32


## Example Complete!

This notebook demonstrated:
- Creating custom Mesh subclasses with additional numpy arrays
- Working with nested dictionaries containing arrays
- Using BaseModel instances with arrays inside dictionaries
- **Nested Packables**: Direct Packable fields are now supported in `save_to_zip`/`load_from_zip`
- **Extract/Reconstruct**: How nested Packables are serialized as `$ref` checksums and reconstructed using Pydantic schema type info
- **Lazy Loading**: Using callable asset fetchers for on-demand loading